# Video File Formats

> A video file format is a type of file format for storing digital video data on a computer system. Video is almost always stored using lossy compression to reduce the file size. *wikipedia*

Normally a video is in a **container** with visual data and audio data. The container can also have many other useful information as metadata, title, subtitles, etc.

In most of the cases, in order to save memory, the visual and audio data are stored with lossy compressions.

More details and comparisons:
 1. [Video File Format](https://en.wikipedia.org/wiki/Video_file_format)
 2. [Comparison of Container Formats](https://en.wikipedia.org/wiki/Comparison_of_video_container_formats)

# Video Data Handling

There are two aspects to consider for accelerated video data handling in the context of machine learning, the availability of hardware accelerated decoding and efficient video frame indicization.

## Hardware Accelerated Decoding

We intentionally focus on decoding here. In fact, to train a LLM/VLM, we need to read TB of data, making accelerated data reading essential to complete trainings in a reasonable time.

We all know the GPU is a powerful piece of hardware that can crunch huge loads of parallel computations very quickly. What many might not know is that the GPU has many specialized cores inside separated from the CUDA cores, and there are video decoding cores (and in many GPUs also encoding cores).

![immagine](assets/nvcodec.jpg)

A tipical setting is to use FFmpeg and decode the video data in CPU before sending it on the GPU for the model training. This can be a bottleneck and to improve it we can compile FFmpeg with NVIDIA GPU acceleration such that FFmpeg can perform hardware accelerated video decoding.

More details on [NVCODEC Docs](https://docs.nvidia.com/video-technologies/video-codec-sdk/12.2/index.html#ffmpeg-with-nvidia-gpu).

##  Indexing Friendly Video Formats

Formats that use keyframes and support random access tend to be more indexing-friendly. These allow for quicker seeking to specific points in the video without having to decode all preceding frames. The efficiency of indexing can also depend on how the video was encoded within the container. Videos with regular keyframe intervals are generally easier to index and seek through.

Some container formats known for good indexing support include:
 - MP4 (MPEG-4 Part 14)
 - MKV (Matroska)
 - AVI (Audio Video Interleave)
 - MOV (QuickTime File Format)

Side Note: It's worth noting that the efficiency of indexing can also depend on the specific implementation of the video processing library or tool being used, not just the inherent properties of the video format itself.

Citations:
[1] https://imagekit.io/blog/video-indexing/
[2] https://indexrusher.com/blog/video-compression-for-seo-formats-and-optimization/
[3] https://www.gumlet.com/learn/video-indexing/
[4] https://www.linkedin.com/pulse/everything-you-need-know-video-indexing-gumlet
[5] https://cloudinary.com/guides/web-performance/video-encoding-how-it-works-formats-best-practices
[6] https://docs.nvidia.com/deeplearning/dali/user-guide/docs/operations/nvidia.dali.fn.readers.video.html
[7] https://docs.nvidia.com/video-technologies/video-codec-sdk/12.0/ffmpeg-with-nvidia-gpu/index.html
[8] https://docs.nvidia.com/deeplearning/dali/user-guide/docs/operations/nvidia.dali.fn.experimental.readers.video.html

## Video Preprocessing

In many datasets we might find videos with different container format, different resolutions, different frame rates. In addition to that, our machine learning training will usually use a standard resolution in input (*e.g.* 256x128).

Assuming you already overcame the challenge of downloading or transferring your dataset to the cluster we can move of to the video preprocessing stage.

To have detailed information about the video we can use
```bash
ffmpeg -i input.mp4
```
We can preprocess our videos with

```bash
ffmpeg -y -vsync 0 -hwaccel cuda -hwaccel_output_format cuda –resize 1280x720 -i input.mp4 -c:a copy -c:v h264_nvenc -b:v 5M output.mp4
```
where we say to ffmpeg to 
 - ```-y```: Overwrite output file without asking for confirmation.
 - ```-vsync 0```: Disable video sync to potentially speed up the process.
 - ```-hwaccel cuda```: Use CUDA hardware acceleration for decoding.
 - ```-hwaccel_output_format cuda```: Set the output format of the hardware accelerated decoder to CUDA, keeping the frames on the GPU.
 - ```–resize 1280x720```: Resize the video to 1280x720 resolution. (Note: This flag seems to be incorrect. The correct flag would be -vf scale=1280:720.)
 - ```-i input.mp4```: Specify the input file.
 - ```-c:a copy```: Copy the audio stream without re-encoding.
 - ```-c:v h264_nvenc```: Use the NVIDIA H.264 encoder for video encoding.
 - ```-b:v 5M```: Set the video bitrate to 5 Mbps.
 - ```output.mp4```: Specify the output file name.
 
We need a script to preprocess all our dataset and not manually do one video at the time. A possible example is the one we provide in [preprocessing.sh](assets/preprocess.sh). You can visualize it with the command below, we replicate the dataset folder structure and then convert all the videos into the new root folder.

In [ ]:
! cat assets/preprocess.sh